Import Libraries

In [25]:
# import libraries
import pandas as pd
import csv
import numpy as np

Load Data


In [26]:
# load data and create dataframe
rawdata = pd.read_csv("https://raw.githubusercontent.com/felicitimilne/bluefall2hwteam17/main/NcstateSinglesStatReport.csv")
indoor_outdoor = pd.read_csv("https://github.com/felicitimilne/bluefall2hwteam17/raw/main/Indoors%20and%20Outdoors.csv")

Data Cleaning & Considerations

In [27]:
# make clean dataset
cleandata = rawdata

# remove white space from player name
cleandata['player'] = cleandata['player'].str.strip()

# remove errors in rally length(those that do not match with the time duration
##### team - how should we deal with this? 
##### my idea is checking the distribution of rally lengths, checking all rally lengths above a certain (unlikely) number, then cross-check with point time duration and the actual footage on Cizr.

# take care of instances where nc state players play against each other - are they counted separately/are they reflected in the data?
# "coach secker: Count them for each of our players as normal. They are counting matches in a players record."
##### team - how should we deal with this?


Adding Seasons to data

In [28]:
# create seasons function

# if month is 1,2,3,4,5,6 --> current season, take year value 
# if month is 7,8,9,10,11,12 --> next season, year value + 1

def find_season(date):
    if int(date.split('/')[0]) < 7:
        return int(date.split('/')[2])
    else:
        return int(date.split('/')[2]) + 1

# adding season column to data

cleandata.insert(loc = 3,
          column = 'season',
          value = '')

cleandata['season'] = cleandata['date'].apply(find_season) 

Create Lists

In [29]:
# create list of player names
players = cleandata['player'].unique()

List of Functions for all columns needed for output data frame

In [32]:
def get_player_name(df, match_id):
    return df[(df['matchId'] == match_id)]['player'].iloc[0]

def get_opp_name(df, match_id):
    return df[(df['matchId'] == match_id)]['opp'].iloc[0]

def get_season(df, match_id):
    return df[(df['matchId'] == match_id)]['season'].iloc[0]

def get_date(df, match_id):
    return df[(df['matchId'] == match_id)]['date'].iloc[0]

def get_match_result(df, match_id):
    
    score_str = df[(df['matchId'] == match_id)]['finalScore'].iloc[0]
    final_score_list_split = score_str.split("|")
    outcome_list = []
    
    for j in range(len(final_score_list_split)):
        
        final_score_list_split2 = final_score_list_split[j].split("-")
        outcomeWin = ""
        
        if (int(final_score_list_split2[0]) == 6 and int(final_score_list_split2[1]) <= 4) or \
        (int(final_score_list_split2[0]) == 7) or \
        (j == 2 and int(final_score_list_split2[0]) == 1 and int(final_score_list_split2[1]) == 0):
            outcomeWin = "NCSU"
            
        elif (int(final_score_list_split2[1]) == 6 and int(final_score_list_split2[0]) <= 4) or \
        (int(final_score_list_split2[1]) == 7) or \
        (j == 2 and int(final_score_list_split2[0]) == 0 and int(final_score_list_split2[1]) == 1):
            outcomeWin = "Opponent"
            
        else:
            outcomeWin = "Incomplete"
        
        outcome_list.append(outcomeWin)    
    
    if outcome_list.count("NCSU") == 2:
        return "Win"
    elif outcome_list.count("Opponent") == 2:
        return "Loss"
    return "Incomplete"

def indoors(match_id):
    if indoor_outdoor[(indoor_outdoor['matchId'] == match_id)]['Indoors'].iloc[0] == "Indoors":
        return True
    else:
        return False

def total_points_won(df, match_id):
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 0)])

def winners(df, match_id):
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 0) & (df['outcome'] == 'Winner')])

def forehand_winners(df, match_id):
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 0) & (df['outcome'] == 'Winner') & (df['shotType'] == 'Forehand')])

def backhand_winners(df, match_id):
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 0) & (df['outcome'] == 'Winner') & (df['shotType'] == 'Backhand')])

def errors_forced(df, match_id): 
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 0) & (df['outcome'] == 'ForcedError')])

def unforced_errors(df, match_id): 
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 1) & (df['outcome'] == 'UnforcedError')])

def unforced_errors_net(df, match_id): # added
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 1) & (df['outcome'] == 'UnforcedError') & (df['errorType'] == 'Net')])

def unforced_errors_long(df, match_id): # added
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 1) & (df['outcome'] == 'UnforcedError') & (df['errorType'] == 'Long')])

def unforced_errors_forehand(df, match_id):     #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 1) & (df['outcome'] == 'UnforcedError') & (df['shotType'] == 'Forehand')])

def unforced_errors_backhand(df, match_id):     #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 1) & (df['outcome'] == 'UnforcedError') & (df['shotType'] == 'Backhand')])

def forced_errors_by_opponent(df, match_id): # added
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 1) & (df['outcome'] == 'ForcedError')])

def forced_error_by_opponent_net(df, match_id): # added
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 1) & (df['outcome'] == 'ForcedError') & (df['errorType'] == 'Net')])

def forced_error_by_opponent_long(df, match_id): # added
    return len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 1) & (df['outcome'] == 'ForcedError') & (df['errorType'] == 'Long')])

def break_points_total(df, match_id):
    return len(df[(df['matchId'] == match_id) & (df['breakPoint'] == True) & (df['server'] == 1)])

def break_points_won(df, match_id):
    return len(df[(df['matchId'] == match_id) & (df['breakPoint'] == True) & (df['server'] == 1) & (df['pointWonBy'] == 0)])

def break_point_percentage(df, match_id):
    if break_points_total(df, match_id) == 0:
        return 0
    return break_points_won(df, match_id) / break_points_total(df, match_id)

def aces(df, match_id):
    return len(df[(df['matchId'] == match_id) & (df['outcome'] == 'Ace') & (df['server'] == 0)])

def double_faults(df, match_id):
    return len(df[(df['matchId'] == match_id) & (df['outcome'] == 'Fault') & (df['server'] == 0)])

def double_faults_error_net(df, match_id): #added
    return len(df[(df['matchId'] == match_id) & (df['outcome'] == 'Fault') & (df['server'] == 0) & (df['errorType'] == 'Net')])

def double_faults_error_long(df, match_id): #added
    return len(df[(df['matchId'] == match_id) & (df['outcome'] == 'Fault') & (df['server'] == 0) & (df['errorType'] == 'Long')])

def total_serves(df, match_id):
    return len(df[(df['matchId'] == match_id) & (df['server'] == 0)])

def first_serves_in(df, match_id):
    return len(df[(df['matchId'] == match_id) & (df['server'] == 0) & (df['firstServeIn'] == True)])

def first_serve_in_percentage(df, match_id):
    return first_serves_in(df, match_id) / total_serves(df, match_id)

def first_serves_won(df, match_id):     #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['server'] == 0) & (df['firstServeIn'] == True) & (df['pointWonBy'] == 0)])

def first_serves_won_percentage(df, match_id):   #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['server'] == 0) & (df['firstServeIn'] == True) & (df['pointWonBy'] == 0)]) / first_serves_in(df, match_id)

def total_second_serves(df, match_id):
    return total_serves(df, match_id) - first_serves_in(df, match_id)

def second_serves_in(df, match_id):
    return total_second_serves(df, match_id) - len(df[(df['matchId'] == match_id) & (df['server'] == 0) & (df['firstServeIn'] == False) & (df['outcome'] == 'Fault')])

def second_serves_in_percentage(df, match_id):
    return second_serves_in(df, match_id) / total_second_serves(df, match_id)

def second_serves_won(df, match_id):    #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['server'] == 0) & (df['firstServeIn'] == False) & (df['pointWonBy'] == 0)])

def second_serves_won_percentage(df, match_id): #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['server'] == 0) & (df['firstServeIn'] == False) & (df['pointWonBy'] == 0)]) / total_second_serves(df, match_id)

def first_serve_returns(df, match_id):  #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['returner'] == 0) & (df['firstServeIn'] == True) & (df['returnInPlay'] == True)])

def first_serve_returns_percentage(df, match_id): #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['returner'] == 0) & (df['firstServeIn'] == True) & (df['returnInPlay'] == True)]) / len(df[(df['matchId'] == match_id) & (df['returner'] == 0) & (df['firstServeIn'] == True)])

def second_serve_returns(df, match_id):     #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['returner'] == 0) & (df['firstServeIn'] == False) & (df['outcome'] != "Fault") & (df['returnInPlay'] == True)])

def second_serve_returns_percentage(df, match_id): #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['returner'] == 0) & (df['firstServeIn'] == False) & (df['outcome'] != "Fault") & (df['returnInPlay'] == True)]) / len(df[(df['matchId'] == match_id) & (df['returner'] == 0) & (df['firstServeIn'] == False) & (df['outcome'] != "Fault")])

def short_rallies_won(df, match_id):    #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['rallyLength'] < 5) & (df['pointWonBy'] == 0)])

def short_rallies_won_percentage(df, match_id):  #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['rallyLength'] < 5) & (df['pointWonBy'] == 0)]) / len(df[(df['matchId'] == match_id) & (df['rallyLength'] < 5)])

def medium_rallies_won(df, match_id):   #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['rallyLength'] >= 5) & (df['rallyLength'] < 9) & (df['pointWonBy'] == 0)])

def medium_rallies_won_percentage(df, match_id):  #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['rallyLength'] >= 5) & (df['rallyLength'] < 9) & (df['pointWonBy'] == 0)]) / len(df[(df['matchId'] == match_id) & (df['rallyLength'] >= 5) & (df['rallyLength'] < 9)])

def long_rallies_won(df, match_id):     #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['rallyLength'] > 8) & (df['pointWonBy'] == 0)])

def long_rallies_won_percentage(df, match_id):  #added by Josh
    return len(df[(df['matchId'] == match_id) & (df['rallyLength'] > 8) & (df['pointWonBy'] == 0)]) / len(df[(df['matchId'] == match_id) & (df['rallyLength'] > 8)])

def service_games_total(df, match_id):
    return len(pd.unique(df[(df['matchId'] == match_id) & (df['server'] == 0) & (df['tiebreaker'] == False)]['game']))

def service_games_won(df, match_id):
    return len(pd.unique(df[(df['matchId'] == match_id) & (df['server'] == 0) & (df['tiebreaker'] == False) & (df['gameWonBy'] == 0)]['game']))

def service_games_won_percentage(df, match_id):
    return service_games_won(df, match_id) / service_games_total(df, match_id)

# aggressive error margin 1 formula = (Winner + Forced Errors)-unforced errors
def aggressive_error_margin_1(df, match_id):
    return (len(df[((df['matchId'] == match_id) & (df['pointWonBy'] == 0)) & ((df['outcome'] == 'Winner') | (df['outcome'] == 'ForcedError'))])) - (len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 1) & (df['outcome'] == 'UnforcedError')]))

# aggressive error margin 2 formula = (Winner + Forced Errors + Aces) - (unforced errors + double faults)
def aggressive_error_margin_2(df, match_id):  #added by Josh
    return (len(df[((df['matchId'] == match_id) & (df['pointWonBy'] == 0)) & ((df['outcome'] == 'Ace') | (df['outcome'] == 'Winner') | (df['outcome'] == 'ForcedError'))])) - (len(df[(df['matchId'] == match_id) & (df['pointWonBy'] == 1) & ((df['outcome'] == 'Fault') | (df['outcome'] == 'UnforcedError'))]))

# First serve performance rating metric formula = (First Serve %)x(first serve win %/100)
def first_serve_performance(df, match_id):
    return first_serve_in_percentage(df, match_id) * first_serves_won(df, match_id)


Create data frame using all the functions


Desired output:

df1: 1 row per player

player name
17+ metrics

df2: 1 row per match (long form, multiple rows per player)

matchid
player, opponent name 
season
match result: win/lose
indoor/outdoor
17+ metrics

In [89]:
# match stats data frame

unique_match_ids = list(cleandata['matchId'].unique())

df = cleandata

completed_frame = {'date' : [get_date(df, x) for x in unique_match_ids],
                    'match_id' : unique_match_ids,
                    'player': [get_player_name(df, x) for x in unique_match_ids],
                    'opponent': [get_opp_name(df, x) for x in unique_match_ids],
                    'season' : [get_season(df, x) for x in unique_match_ids],
                    'match_result' : [get_match_result(df, x) for x in unique_match_ids],
                    'indoors' : [indoors(x) for x in unique_match_ids],
                    'total_points_won' : [total_points_won(df, x) for x in unique_match_ids],
                    'winners' : [winners(df, x) for x in unique_match_ids],
                    'forehand_winner' : [forehand_winners(df, x) for x in unique_match_ids],
                    'backhand_winner' : [backhand_winners(df, x) for x in unique_match_ids],
                    'errors_forced' : [errors_forced(df, x) for x in unique_match_ids],
                    'unforced_errors' : [unforced_errors(df, x) for x in unique_match_ids],
                    'unforced_errors_forehand' : [unforced_errors_forehand(df, x) for x in unique_match_ids],
                    'unforced_errors_backhand' : [unforced_errors_backhand(df, x) for x in unique_match_ids],
                    'forced_errors_by_opponent' : [forced_errors_by_opponent(df, x) for x in unique_match_ids],
                    'break_points_total' : [break_points_total(df, x) for x in unique_match_ids],
                    'break_points_won' : [break_points_won(df, x) for x in unique_match_ids],
                    'break_points_percentage' : [break_point_percentage(df, x) for x in unique_match_ids],
                    'aces' : [aces(df, x) for x in unique_match_ids],
                    'double_faults' : [double_faults(df, x) for x in unique_match_ids],
                    'total_serves' : [total_serves(df, x) for x in unique_match_ids],
                    'first_serves_in' : [first_serves_in(df, x) for x in unique_match_ids],
                    'first_serve_in_percentage' : [first_serve_in_percentage(df, x) for x in unique_match_ids],
                    'first_serves_won' : [first_serves_won(df, x) for x in unique_match_ids],
                    'first_serves_won_percentage' : [first_serves_won_percentage(df, x) for x in unique_match_ids],
                    'total_second_serves' : [total_second_serves(df, x) for x in unique_match_ids],
                    'second_serves_in' : [second_serves_in(df, x) for x in unique_match_ids],
                    'second_serves_in_percentage' : [second_serves_in_percentage(df, x) for x in unique_match_ids],
                    'second_serves_won' : [second_serves_won(df, x) for x in unique_match_ids],
                    'second_serves_won_percentage' : [second_serves_won_percentage(df, x) for x in unique_match_ids],
                    'first_serve_returns' : [first_serve_returns(df, x) for x in unique_match_ids],
                    'first_serve_returns_percentage' : [first_serve_returns_percentage(df, x) for x in unique_match_ids],
                    'second_serve_returns' : [second_serve_returns(df, x) for x in unique_match_ids],
                    'second_serve_returns_percentage' : [second_serve_returns_percentage(df, x) for x in unique_match_ids],
                    'short_rallies_won' : [short_rallies_won(df, x) for x in unique_match_ids],
                    'short_rallies_won_percentage' : [short_rallies_won_percentage(df, x) for x in unique_match_ids],
                    'medium_rallies_won' : [medium_rallies_won(df, x) for x in unique_match_ids],
                    'medium_rallies_won_percentage' : [medium_rallies_won_percentage(df, x) for x in unique_match_ids],
                    'long_rallies_won' : [long_rallies_won(df, x) for x in unique_match_ids],
                    'long_rallies_won_percentage' : [long_rallies_won_percentage(df, x) for x in unique_match_ids],
                    'service_games_total' : [service_games_total(df, x) for x in unique_match_ids],
                    'service_games_won' : [service_games_won(df, x) for x in unique_match_ids],
                    'service_games_won_percentage' : [service_games_won_percentage(df, x) for x in unique_match_ids],
                    'unforced_errors_net' : [unforced_errors_net(df, x) for x in unique_match_ids],
                    'unforced_errors_long' : [unforced_errors_long(df, x) for x in unique_match_ids],
                    'forced_errors_by_opponent_net' : [forced_error_by_opponent_net(df, x) for x in unique_match_ids],
                    'forced_errors_by_opponent_long' : [forced_error_by_opponent_long(df, x) for x in unique_match_ids],
                    'double_faults_error_net' : [double_faults_error_net(df, x) for x in unique_match_ids],
                    'double_faults_error_long' : [double_faults_error_long(df, x) for x in unique_match_ids],
                    'aggressive_error_margin_1' : [aggressive_error_margin_1(df, x) for x in unique_match_ids],
                    'aggressive_error_margin_2' : [aggressive_error_margin_2(df, x) for x in unique_match_ids],
                    'first_serve_performance' : [first_serve_performance(df, x) for x in unique_match_ids]}


match_stats = pd.DataFrame(data = completed_frame)



In [90]:
def rank(name: str, df):
    while name not in df.columns:
        name = input("Invalid input, try one of these names\n" + '\n'.join(list(df.columns)) + "\t")
    neg_stat = name in ["double_faults", "unforced_errors"]
    sorted_vals = df.sort_values(name, ascending = neg_stat)
    ranked_dict = {list(sorted_vals.index)[i]:i + 1 for i in range(len(list(sorted_vals.index)))}
    sorted_list = sorted(ranked_dict.items())
    return {sorted_list[i][0]:sorted_list[i][1] for i in range(len(sorted_list))}
    
players = match_stats["player"]
msp = match_stats[match_stats.columns[7:len(match_stats.columns)]]
msp = pd.concat([players, msp], axis = 1)
msp
gms = msp.groupby(by = "player").mean()
gms
rank("errors_forced", gms)

{'Abigail Rencheli': 6,
 'Adriana Reami': 7,
 'Alana Smith': 9,
 'Amanda Rebol': 17,
 'Amelia Rajecki': 3,
 'Anna Rogers': 12,
 'Bianca Moldovan': 16,
 'Claudia Wiktorin': 10,
 'Dylan Cannon': 11,
 'Jaeda Daniel': 4,
 'Lana Mavor': 15,
 'Lexi Keberle': 14,
 'Liz Norman': 13,
 'Marta Gonzalez': 8,
 'Nell Miller': 2,
 'Priska Nugroho': 5,
 'Sara Nayar': 1,
 'Sophie Abrams': 18}

In [91]:
import collections

def agg_rank(df):
    metrics = ["aggressive_error_margin_2", "aces", "break_points_percentage", \
              "second_serve_returns_percentage", "errors_forced", \
              "short_rallies_won_percentage", "medium_rallies_won_percentage", "winners"]
    
    rank_list = []
    for i in range(len(metrics)):
        rank_list.append(rank(metrics[i], df))
        
    counter = collections.Counter()
    for d in rank_list:
        counter.update(d)
        
    ar_dict = dict(counter)
    return {(list(ar_dict.keys()))[j]:round(((list(ar_dict.values()))[j] / len(metrics)), 2) for j in range(len(list(ar_dict.items())))}
    

def ret_rank(df):
    metrics = ["break_points_percentage", "first_serve_returns_percentage", \
                "second_serve_returns_percentage", "short_rallies_won_percentage", "winners"]
    rank_list = []
    for i in range(len(metrics)):
        rank_list.append(rank(metrics[i], df))
        
    counter = collections.Counter()
    for d in rank_list:
        counter.update(d)
        
    ar_dict = dict(counter)
    return {(list(ar_dict.keys()))[j]:round(((list(ar_dict.values()))[j] / len(metrics)), 2) for j in range(len(list(ar_dict.items())))}

def gs_rank(df):
    metrics = ["aggressive_error_margin_2", "break_points_percentage", \
               "first_serve_returns_percentage", "long_rallies_won", "unforced_errors", \
              "second_serve_returns_percentage", "errors_forced", \
              "short_rallies_won_percentage", "medium_rallies_won_percentage", "winners"]
    
    rank_list = []
    for i in range(len(metrics)):
        rank_list.append(rank(metrics[i], df))
        
    counter = collections.Counter()
    for d in rank_list:
        counter.update(d)
        
    ar_dict = dict(counter)
    return {(list(ar_dict.keys()))[j]:round(((list(ar_dict.values()))[j] / len(metrics)), 2) for j in range(len(list(ar_dict.items())))}

def serve_rank(df):
    metrics = ["aces", "double_faults", "first_serve_in_percentage", \
               "first_serve_performance", "first_serves_won_percentage", \
               "second_serves_in_percentage", "second_serves_won_percentage", \
               "service_games_won_percentage"]
    
    rank_list = []
    for i in range(len(metrics)):
        rank_list.append(rank(metrics[i], df))
        
    counter = collections.Counter()
    for d in rank_list:
        counter.update(d)
        
    ar_dict = dict(counter)
    return {(list(ar_dict.keys()))[j]:round(((list(ar_dict.values()))[j] / len(metrics)), 2) for j in range(len(list(ar_dict.items())))}

agg_rank(gms)

{'Abigail Rencheli': 8.88,
 'Adriana Reami': 7.25,
 'Alana Smith': 6.88,
 'Amanda Rebol': 11.88,
 'Amelia Rajecki': 11.12,
 'Anna Rogers': 8.25,
 'Bianca Moldovan': 8.0,
 'Claudia Wiktorin': 7.0,
 'Dylan Cannon': 9.12,
 'Jaeda Daniel': 8.12,
 'Lana Mavor': 10.75,
 'Lexi Keberle': 9.5,
 'Liz Norman': 11.38,
 'Marta Gonzalez': 13.25,
 'Nell Miller': 7.0,
 'Priska Nugroho': 10.0,
 'Sara Nayar': 9.12,
 'Sophie Abrams': 13.5}

In [92]:
gs_rank(gms)

{'Abigail Rencheli': 9.8,
 'Adriana Reami': 7.3,
 'Alana Smith': 9.4,
 'Amanda Rebol': 9.0,
 'Amelia Rajecki': 12.3,
 'Anna Rogers': 8.4,
 'Bianca Moldovan': 8.5,
 'Claudia Wiktorin': 7.3,
 'Dylan Cannon': 9.9,
 'Jaeda Daniel': 8.6,
 'Lana Mavor': 10.0,
 'Lexi Keberle': 8.0,
 'Liz Norman': 11.4,
 'Marta Gonzalez': 12.3,
 'Nell Miller': 6.9,
 'Priska Nugroho': 8.9,
 'Sara Nayar': 9.9,
 'Sophie Abrams': 13.1}

In [93]:
ret_rank(gms)

{'Abigail Rencheli': 9.6,
 'Adriana Reami': 7.4,
 'Alana Smith': 7.8,
 'Amanda Rebol': 8.2,
 'Amelia Rajecki': 13.6,
 'Anna Rogers': 6.2,
 'Bianca Moldovan': 5.8,
 'Claudia Wiktorin': 8.4,
 'Dylan Cannon': 10.6,
 'Jaeda Daniel': 8.4,
 'Lana Mavor': 9.4,
 'Lexi Keberle': 8.8,
 'Liz Norman': 10.2,
 'Marta Gonzalez': 13.8,
 'Nell Miller': 6.8,
 'Priska Nugroho': 12.2,
 'Sara Nayar': 9.6,
 'Sophie Abrams': 14.2}

In [94]:
serve_rank(gms)

{'Abigail Rencheli': 11.75,
 'Adriana Reami': 6.25,
 'Alana Smith': 5.5,
 'Amanda Rebol': 13.38,
 'Amelia Rajecki': 8.75,
 'Anna Rogers': 5.25,
 'Bianca Moldovan': 5.0,
 'Claudia Wiktorin': 6.62,
 'Dylan Cannon': 12.88,
 'Jaeda Daniel': 12.88,
 'Lana Mavor': 8.62,
 'Lexi Keberle': 6.25,
 'Liz Norman': 10.5,
 'Marta Gonzalez': 13.62,
 'Nell Miller': 11.25,
 'Priska Nugroho': 9.75,
 'Sara Nayar': 12.62,
 'Sophie Abrams': 10.12}

In [95]:
agg_rank_list = []
gs_rank_list = []
ret_rank_list = []
serve_rank_list = []
for i in list(match_stats["player"]):
    agg_rank_list.append(agg_rank(gms)[i])
    gs_rank_list.append(gs_rank(gms)[i])
    ret_rank_list.append(ret_rank(gms)[i])
    serve_rank_list.append(serve_rank(gms)[i])

ms_temp = pd.concat([match_stats, pd.Series(agg_rank_list), \
                     pd.Series(gs_rank_list), pd.Series(ret_rank_list), \
                     pd.Series(serve_rank_list)], axis = 1)



In [96]:
ms_temp.head(20)
ms_temp = ms_temp.rename(columns = {list(ms_temp.columns)[-4]:"aggressive_rank", \
                            list(ms_temp.columns)[-3]:"groundstroke_rank", \
                            list(ms_temp.columns)[-2]:"return_rank", \
                            list(ms_temp.columns)[-1]:"serve_rank"})

In [97]:
ms_temp.head()

,date,match_id,player,opponent,season,match_result,indoors,total_points_won,winners,forehand_winner,...,forced_errors_by_opponent_long,double_faults_error_net,double_faults_error_long,aggressive_error_margin_1,aggressive_error_margin_2,first_serve_performance,aggressive_rank,groundstroke_rank,return_rank,serve_rank
0,2/13/2022,62150d674500004a00cc8886,Priska Nugroho,Ansari,2022,Incomplete,True,55,9,5,...,9,3,1,0,-3,6.000000,10.0,8.9,12.2,9.75
1,2/13/2022,6217b1084200004b002ff9b4,Priska Nugroho,Vidmanova,2022,Win,True,60,32,8,...,1,0,1,13,16,11.880000,10.0,8.9,12.2,9.75
2,2/13/2022,6218fc1f420000650a2ff9f0,Priska Nugroho,Tran,2022,Loss,True,34,10,3,...,8,0,1,3,2,10.588235,10.0,8.9,12.2,9.75
3,2/13/2022,62150dc64500004900cc8887,Nell Miller,Flach,2022,Win,True,58,5,3,...,4,5,0,4,4,12.727273,7.0,6.9,6.8,11.25
4,2/13/2022,6217b1d642000033002ff9b6,Nell Miller,Hertel,2022,Win,True,58,13,3,...,1,3,1,-3,-3,9.219512,7.0,6.9,6.8,11.25


In [98]:
ms_temp.round(2).to_csv('/Users/noahjohnson/Downloads/bluefall2hwteam17/match_stats_ranks.csv', index = False)

In [30]:
# check data types

cleandata.dtypes

matchId           object
pointNum           int64
date              object
season             int64
matchName         object
player            object
player2          float64
opp               object
opp2             float64
set                int64
setId             object
game               int64
gameId            object
setsWon            int64
oppSetsWon         int64
gamesWon           int64
oppGamesWon        int64
pointsWon          int64
oppPointsWon       int64
tiebreaker          bool
breakPoint          bool
gamePoint           bool
noAd                bool
server             int64
returner           int64
score              int64
oppScore           int64
serverScore        int64
returnerScore      int64
firstServeIn        bool
outcome           object
errorType         object
shotType          object
rallyLength        int64
pointWonBy         int64
gameWonBy          int64
endingPlayer       int64
returnInPlay        bool
matchType         object
setTo              int64


In [31]:
cleandata.head()

,matchId,pointNum,date,season,matchName,player,player2,opp,opp2,set,...,gameWonBy,endingPlayer,returnInPlay,matchType,setTo,adType,startTime,stopTime,finalScore,matchLink
0,62150d674500004a00cc8886,10000,2/13/2022,2022,NC State vs Auburn,Priska Nugroho,NaN,Ansari,NaN,1,...,0,2,True,Singles,6,NoAd,262.102981,275.332223,6-7|1-2,https://www.cizrtennis.com/userMatches//watch/...
1,62150d674500004a00cc8886,10001,2/13/2022,2022,NC State vs Auburn,Priska Nugroho,NaN,Ansari,NaN,1,...,0,0,True,Singles,6,NoAd,313.902541,318.623684,6-7|1-2,https://www.cizrtennis.com/userMatches//watch/...
2,62150d674500004a00cc8886,10002,2/13/2022,2022,NC State vs Auburn,Priska Nugroho,NaN,Ansari,NaN,1,...,0,2,True,Singles,6,NoAd,341.093521,351.803521,6-7|1-2,https://www.cizrtennis.com/userMatches//watch/...
3,62150d674500004a00cc8886,10003,2/13/2022,2022,NC State vs Auburn,Priska Nugroho,NaN,Ansari,NaN,1,...,0,0,False,Singles,6,NoAd,371.835772,375.316415,6-7|1-2,https://www.cizrtennis.com/userMatches//watch/...
4,62150d674500004a00cc8886,10004,2/13/2022,2022,NC State vs Auburn,Priska Nugroho,NaN,Ansari,NaN,1,...,0,2,True,Singles,6,NoAd,397.237456,407.116616,6-7|1-2,https://www.cizrtennis.com/userMatches//watch/...


Max-Min normalize the radar stats

In [34]:
match_stats["winners_norm"] = (match_stats["winners"]-min(match_stats["winners"]))/(max(match_stats["winners"])-min(match_stats["winners"]))

match_stats["aces_norm"] = (match_stats["aces"]-min(match_stats["aces"]))/(max(match_stats["aces"])-min(match_stats["aces"]))

match_stats["first_serve_performance_norm"] = (match_stats["first_serve_performance"]-min(match_stats["first_serve_performance"]))/(max(match_stats["first_serve_performance"])-min(match_stats["first_serve_performance"]))

match_stats["errors_forced_norm"] = (match_stats["errors_forced"]-min(match_stats["errors_forced"]))/(max(match_stats["errors_forced"])-min(match_stats["errors_forced"]))

match_stats["aggressive_error_margin_2_norm"] = (match_stats["aggressive_error_margin_2"]-min(match_stats["aggressive_error_margin_2"]))/(max(match_stats["aggressive_error_margin_2"])-min(match_stats["aggressive_error_margin_2"]))

match_stats["break_points_percentage"] = match_stats["break_points_won"]/match_stats["break_points_total"]

match_stats["break_points_percentage_norm"] = (match_stats["break_points_percentage"]-min(match_stats["break_points_percentage"]))/(max(match_stats["break_points_percentage"])-min(match_stats["break_points_percentage"]))



Get the averages for each player and write out the radar csv

In [40]:
radar_stats = match_stats.groupby(by="player").mean()[["winners_norm","aces_norm", "first_serve_performance_norm",
             "errors_forced_norm", "aggressive_error_margin_2_norm", "break_points_percentage_norm"]]

In [41]:
radar_stats.to_csv('C:/HWRepo/radar_tennis_test.csv', index = True)